In [1]:
import pandas as pd

df = pd.read_csv(
    "noticias_unificadas.tsv",
    encoding="utf-8",
    sep="\t",
    dtype={"fecha": "string", "titulo": "string", "contenido": "string", "seccion": "string", "link": "string"},
    quoting=0,
    na_filter=False
)

In [2]:
topicos = df['seccion'].value_counts()
print(f"\nTotal de tópicos únicos: {df['seccion'].nunique()}")


Total de tópicos únicos: 7


In [3]:
df.head()

,fecha,titulo,contenido,seccion,link
0,2025-11-09,Jueces rechazan intento de afectación a la ind...,"Desde la ciudad de Tacna, jueces y juezas de t...",Política,https://diariocorreo.pe/politica/jueces-rechaz...
1,2025-11-09,Liga 1: Lo gritan los “Churres” y todo el pueb...,Alianza Atlético le sacó lustre a su clasifica...,Deportes,https://diariocorreo.pe/deportes/alianza-atlet...
2,2025-11-09,Proponen sancionar con hasta 10 años de cárcel...,"La congresista Elizabeth Medina Hermosillo, de...",Política,https://diariocorreo.pe/politica/proponen-sanc...
3,2025-11-09,Este lunes inicia la semana de representación ...,Desde este lunes 10 hasta el viernes 14 de nov...,Política,https://diariocorreo.pe/politica/este-lunes-in...
4,2025-11-09,Selección peruana evalúa reprogramación de par...,La Federación Peruana de Fútbol (FPF) informó ...,Deportes,https://diariocorreo.pe/deportes/seleccion-per...


In [4]:
import nltk
from nltk import trigrams
from nltk.tokenize import sent_tokenize, word_tokenize


In [5]:
nltk.data.find("tokenizers/punkt")

FileSystemPathPointer('/Users/joelibaceta/nltk_data/tokenizers/punkt')

In [6]:
nltk.data.find("tokenizers/punkt_tab")

FileSystemPathPointer('/Users/joelibaceta/nltk_data/tokenizers/punkt_tab')

In [7]:
from typing import Dict, List, Tuple

def tokenize_es(text: str) -> list[list[str]]:
    if not isinstance(text, str) or not text.strip():
        return []
    sentences = sent_tokenize(text, language="spanish")
    tokenized = []
    for s in sentences:
        toks = word_tokenize(s, language="spanish")
        # lower only alphabetic tokens, keep punctuation as-is
        toks = [t.lower() if t.isalpha() else t for t in toks]
        if toks:
            tokenized.append(toks)
    return tokenized

In [9]:
from collections import defaultdict

def load_corpus(df: pd.DataFrame, by_category: bool = False) -> Dict[str, List[List[str]]]:
    buckets: Dict[str, List[List[str]]] = defaultdict(list)
    
    for _, row in df.iterrows():
        categoria = (row.get("seccion") or "").strip()
        noticia = row.get("contenido")
        
        if not noticia or not isinstance(noticia, str):
            continue
            
        sents_toks = tokenize_es(noticia)
        
        if by_category and categoria:
            for s in sents_toks:
                if s:
                    buckets[categoria].append(s)
        else:
            for s in sents_toks:
                if s:
                    buckets["_GLOBAL"].append(s)
    
    return buckets

In [10]:
corpus_por_categoria = load_corpus(df, by_category=True)
corpus_global = load_corpus(df, by_category=False)

In [11]:
def train_trigrams(tokenized_sentences: List[List[str]]):
    """
      model[(w1,w2)][w3] = prob
    """
    model = defaultdict(lambda: defaultdict(float))
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            model[(w1, w2)][w3] += 1.0

    for w1w2 in model:
        total = sum(model[w1w2].values())
        if total > 0:
            for w3 in model[w1w2]:
                model[w1w2][w3] /= total
    return model

In [12]:
model_global = train_trigrams(corpus_global["_GLOBAL"])
len(model_global)

2274244

In [13]:
models_por_categoria = {}

for categoria, oraciones in corpus_por_categoria.items():
    print(f"Entrenando modelo para: {categoria} ({len(oraciones)} oraciones)")
    models_por_categoria[categoria] = train_trigrams(oraciones)


Entrenando modelo para: Política (164273 oraciones)
Entrenando modelo para: Deportes (64217 oraciones)
Entrenando modelo para: Deportes (64217 oraciones)
Entrenando modelo para: Espectáculos (95589 oraciones)
Entrenando modelo para: Espectáculos (95589 oraciones)
Entrenando modelo para: Cultura (82146 oraciones)
Entrenando modelo para: Cultura (82146 oraciones)
Entrenando modelo para: Economía (56308 oraciones)
Entrenando modelo para: Economía (56308 oraciones)
Entrenando modelo para: Mundo (81657 oraciones)
Entrenando modelo para: Mundo (81657 oraciones)
Entrenando modelo para: Policiales (50687 oraciones)
Entrenando modelo para: Policiales (50687 oraciones)


In [14]:
from typing import Optional
import random

def sample_next(model: dict, w1: str, w2: str) -> Optional[str]:
    dist = model.get((w1, w2), {})
    if not dist:
        for ctx in [(None, w2), (w1, None), (None, None)]:
            dist = model.get(ctx, {})
            if dist:
                break
        if not dist:
            return None

    r = random.random()
    acc = 0.0
    for w3, p in dist.items():
        acc += p
        if acc >= r:
            return w3
    return next(iter(dist.keys()))

In [39]:
def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    text: List[str] = [seeds[0], seeds[1]]
    sentence_finished = False

    while not sentence_finished and len(text) < max_len + 2:
        w3 = sample_next(model, text[-2], text[-1])
        text.append(w3)
        if text[-2:] == [None, None] or w3 is None:
            sentence_finished = True

    sentence = " ".join([t for t in text if t])
    sentence = sentence.strip()
    if sentence and sentence[-1] not in ".!?":
        sentence += "."
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
    return sentence

In [40]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    return " ".join(generate_sentence(model, seeds=seeds) for _ in range(n_sentences))

In [41]:
print(generate_paragraph(model_global, n_sentences=3, seeds=("La", "Policia", )))

La Policia ojalá que los cálculos del usda estimó una tasa de crecimiento y la primera en dar un paso firme hacia la ventana . La Policia `` esta organización '' . La Policia es de dos premios internacionales , como dijo un testigo a la escritora peruana : quita estrés – two broders + salsipuedes brewing co ( new york times publicó el.


In [42]:
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=4, seeds=("El", "equipo")))

El equipo juvenil mixto ( natación y haber encontrado un equipo argentino para sellar la clasificación general , por copa libertadores y la paradeportista , niurka callupe ( femenino ) ? El equipo que se armó en colombia , césar vallejo su desvinculación del -ahora- exdirector técnico de la derrota por ippon ante el sampdoria por la semifinal . El equipo que cayó ante su próximo desafío será visitar a la primera dificultad logística que enfrentará al ganador . El equipo libre ¿Qué es el principal objetivo es campeonar .


In [43]:
print("\nPOLÍTICA:")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=3, seeds=("El", "presidente")))


POLÍTICA:
El presidente , que elimina movimientos regionales y locales , abarrotaban plazas y todo lo que aumenta más cada año se sortearán 842,000 miembros de mi cliente y le hemos cambiado de. El presidente de la colisión entre una y dos bomberos lesionadoshuancayo : cajera es detenida por apoderarse de S/26 milxi jinping : “ ahora él es quien investiga y que , a. El presidente alan garcía , un espectáculo de conflicto ” , pérez no querían perder más del 41 , y el bloque democrático popular , avanza país , y humberto miñán almanza.


In [ ]:

def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:

    # Normalizar seeds a minúsculas si no son None
    w1 = seeds[0].lower() if seeds[0] else None
    w2 = seeds[1].lower() if seeds[1] else None
    
    # Iniciar la secuencia
    text: List[str] = []
    if w1:
        text.append(w1)
    if w2:
        text.append(w2)
    
    # Si no hay seeds, empezar desde el inicio de oración
    if not text:
        w1, w2 = None, None
    elif len(text) == 1:
        w1, w2 = None, text[0]
    else:
        w1, w2 = text[-2], text[-1]
    
    sentence_finished = False
    
    while not sentence_finished and len(text) < max_len:
        w3 = sample_next(model, w1, w2)
        
        # Terminar si encontramos el marcador de fin o no hay siguiente palabra
        if w3 is None:
            sentence_finished = True
        else:
            text.append(w3)
            w1, w2 = w2, w3
    
    # Construir la oración
    sentence = " ".join([t for t in text if t])
    sentence = sentence.strip()
    
    # Agregar punto final si no tiene puntuación
    if sentence and sentence[-1] not in ".!?":
        sentence += "."
    
    # Capitalizar primera letra
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
    
    return sentence

In [47]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    """
    Genera un párrafo de múltiples oraciones.
    Solo usa las seeds para la primera oración.
    """
    sentences = []
    # Primera oración usa las seeds
    sentences.append(generate_sentence(model, seeds=seeds))
    
    # Resto de oraciones sin seeds (comienzan naturalmente)
    for _ in range(n_sentences - 1):
        sentences.append(generate_sentence(model, seeds=(None, None)))
    
    return " ".join(sentences)

In [49]:
print(generate_paragraph(model_global, n_sentences=3, seeds=("La", "policía")))

La policía encontró varios casquillos de bala . Varias de estas , el ministerio público . Era niño , al igual que ahora lo hace a través del tiempo , atziri se desplomó sobre el exgobernador regional de ayacucho ? para empezar , solo se regulariza.


In [52]:
print("DEPORTES - Seeds: ('El', 'equipo')")
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=4, seeds=("El", "equipo")))

DEPORTES - Seeds: ('El', 'equipo')
El equipo que cambien el “ submarino amarillo ’ saca 6 puntos en lo individual y persecución por equipos . Perú21 epaper . Directo al mundial 2026 . Búscanos en yape !


In [55]:
print("POLÍTICA - Seeds: ('El', 'presidente')")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=3, seeds=("El", "presidente")))

POLÍTICA - Seeds: ('El', 'presidente')
El presidente de jp en el escenario internacional , y alucinando un encuentro con pamela lópez en discoteca durante concierto de la denunciante.La denuncia fue formulada formalmente por la contraloría general viene. Estar a tu casa como garantía de motivación , ya que tiene categoría de distinción para diferenciar entre los diferentes poderes del estado , ya que anteriormente había tenido expresiones. El texto sustitutorio del proyecto especial regional pasto grande , porque el 24 de febrero , ecuador , según ‘ panorama ’ , boluarte aseguró que su gestión busca potenciar.


In [57]:
print("MODELO GLOBAL - Sin seeds (inicio natural)")
print(generate_paragraph(model_global, n_sentences=3, seeds=(None, None)))

MODELO GLOBAL - Sin seeds (inicio natural)
¡Ahora disponible en yape ! ¿QUIÉN es yessenia lozano millones , aproximadamente 120 millones ) . Zelenski afirmó el consejo empresarial Peruano-Japonés ( cepeja ) , natalia salas confesó que su presunto cómplice fue atrapado por la octava jornada de las devociones más poderosas para inculcar.


## 📊 Análisis: Combinaciones más frecuentes (Trigramas)

In [62]:
from collections import Counter

def get_most_frequent_trigrams(tokenized_sentences: List[List[str]], top_n: int = 20, exclude_padding: bool = True):
    """
    Identifica los trigramas más frecuentes en el corpus.
    
    Args:
        tokenized_sentences: Lista de oraciones tokenizadas
        top_n: Número de trigramas a retornar
        exclude_padding: Si True, excluye trigramas que contienen None (padding)
    
    Returns:
        Lista de tuplas ((w1, w2, w3), frecuencia)
    """
    trigram_counts = Counter()
    
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            # Opcionalmente excluir trigramas con padding
            if exclude_padding and (w1 is None or w2 is None or w3 is None):
                continue
            trigram_counts[(w1, w2, w3)] += 1
    
    return trigram_counts.most_common(top_n)

In [ ]:
print("TRIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL")

top_trigrams_global = get_most_frequent_trigrams(corpus_global["_GLOBAL"], top_n=30)

for i, ((w1, w2, w3), freq) in enumerate(top_trigrams_global, 1):
    w1_str = f"'{w1}'" if w1 else "None"
    w2_str = f"'{w2}'" if w2 else "None"
    w3_str = f"'{w3}'" if w3 else "None"
    
    print(f"{i:2}. ({w1_str:>15}, {w2_str:>15}, {w3_str:>15}) → {freq:>5} veces")

print(f"\nTotal de trigramas únicos: {len(set(trigrams(sum(corpus_global['_GLOBAL'], []), pad_left=True, pad_right=True)))}")

TRIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL


In [ ]:
# Analizar por categoría
print("\n" + "=" * 70)
print("TRIGRAMAS MÁS FRECUENTES POR CATEGORÍA")
print("=" * 70)

for categoria in ["Deportes", "Política", "Economía"]:
    if categoria in corpus_por_categoria:
        print(f"\n📂 {categoria.upper()}")
        print("-" * 70)
        
        top_trigrams = get_most_frequent_trigrams(
            corpus_por_categoria[categoria], 
            top_n=10, 
            exclude_padding=True
        )
        
        for i, ((w1, w2, w3), freq) in enumerate(top_trigrams, 1):
            print(f"  {i:2}. ({w1}, {w2}, {w3}) → {freq} veces")

### Visualización de trigramas frecuentes

In [ ]:
import matplotlib.pyplot as plt

def plot_top_trigrams(trigrams_list, title="Top Trigramas", top_n=15):
    """
    Visualiza los trigramas más frecuentes en un gráfico de barras.
    """
    # Tomar solo los top_n
    trigrams_list = trigrams_list[:top_n]
    
    # Preparar datos
    labels = [f"{w1} {w2} {w3}" for (w1, w2, w3), _ in trigrams_list]
    frequencies = [freq for _, freq in trigrams_list]
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = ax.barh(range(len(labels)), frequencies, color='steelblue', alpha=0.8)
    
    ax.set_yticks(range(len(labels)))
    ax.set_yticklabels(labels, fontsize=10)
    ax.set_xlabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    
    # Agregar valores al final de las barras
    for i, (bar, freq) in enumerate(zip(bars, frequencies)):
        ax.text(freq + 0.5, i, str(freq), va='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualizar trigramas del corpus global
plot_top_trigrams(top_trigrams_global, title="Top 15 Trigramas - Corpus Global", top_n=15)

In [ ]:
# Comparación entre categorías
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Trigramas Más Frecuentes por Categoría', fontsize=16, fontweight='bold', y=0.995)

categorias_plot = ["Deportes", "Política", "Economía", "Internacional"]
axes_flat = axes.flatten()

for idx, categoria in enumerate(categorias_plot):
    if categoria in corpus_por_categoria:
        top_cat = get_most_frequent_trigrams(
            corpus_por_categoria[categoria], 
            top_n=10, 
            exclude_padding=True
        )
        
        labels = [f"{w1} {w2} {w3}" for (w1, w2, w3), _ in top_cat]
        frequencies = [freq for _, freq in top_cat]
        
        ax = axes_flat[idx]
        bars = ax.barh(range(len(labels)), frequencies, color='coral', alpha=0.7)
        ax.set_yticks(range(len(labels)))
        ax.set_yticklabels(labels, fontsize=9)
        ax.set_xlabel('Frecuencia', fontsize=10)
        ax.set_title(f'📂 {categoria}', fontsize=12, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(axis='x', alpha=0.3)
        
        # Valores en las barras
        for i, (bar, freq) in enumerate(zip(bars, frequencies)):
            ax.text(freq + 0.3, i, str(freq), va='center', fontsize=8)
    else:
        axes_flat[idx].text(0.5, 0.5, f'Categoría "{categoria}"\nno disponible', 
                           ha='center', va='center', transform=axes_flat[idx].transAxes)
        axes_flat[idx].set_xticks([])
        axes_flat[idx].set_yticks([])

plt.tight_layout()
plt.show()

### Análisis adicional: Bigramas más frecuentes

In [ ]:
from nltk import bigrams

def get_most_frequent_bigrams(tokenized_sentences: List[List[str]], top_n: int = 20, exclude_padding: bool = True):
    """
    Identifica los bigramas más frecuentes en el corpus.
    """
    bigram_counts = Counter()
    
    for sent in tokenized_sentences:
        for w1, w2 in bigrams(sent, pad_left=True, pad_right=True):
            if exclude_padding and (w1 is None or w2 is None):
                continue
            bigram_counts[(w1, w2)] += 1
    
    return bigram_counts.most_common(top_n)

# Analizar bigramas
print("=" * 70)
print("BIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL")
print("=" * 70)

top_bigrams = get_most_frequent_bigrams(corpus_global["_GLOBAL"], top_n=30, exclude_padding=True)

for i, ((w1, w2), freq) in enumerate(top_bigrams, 1):
    print(f"{i:2}. ('{w1}', '{w2}') → {freq:>5} veces")

### 📈 Interpretación de resultados

Los trigramas más frecuentes te muestran:
1. **Patrones comunes** en el lenguaje periodístico español
2. **Frases típicas** que se repiten en las noticias
3. **Contextos específicos** de cada categoría (deportes, política, etc.)

Esto es útil para:
- Entender qué combinaciones de palabras son más naturales
- Mejorar la generación de texto (el modelo aprende de estas frecuencias)
- Identificar vocabulario característico de cada temática